In [5]:
# IMPORT
import pandas as pd
import statistics as st
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

# DATAS
train=pd.read_csv('train.csv',sep=',')
test=pd.read_csv('test.csv',sep=',')
train.set_index('PassengerId',inplace=True,drop=True)
#print(train.columns)
train.dtypes
#print(train.size)
#print(train.ndim)

####
def parse_model_4(X):
    target = X.Survived
    X['title'] = X.Name.map(lambda x : x.split(',')[1].split('.')[0])
    X['surname'] = X.Name.map(lambda x : '(' in x)
    X['Cabin'] = X.Cabin.map(lambda x : x[0] if not pd.isnull(x) else -1)
    to_dummy = ['Pclass', 'Sex', 'title', 'Embarked', 'Cabin']
    for dum in to_dummy :
        split_feature = pd.get_dummies(X[dum],prefix='split_'+dum)
        X = X.join(split_feature)
        del X[dum]
    X['Age'] = X.Age.fillna(X.Age.median())
    X['is_child'] = X.Age < 16
    to_del = ['Name', 'Survived', 'Ticket']
    for col in to_del : del X[col]
    return X, target
def parse_model_test(X):
    X['title'] = X.Name.map(lambda x : x.split(',')[1].split('.')[0])
    X['surname'] = X.Name.map(lambda x : '(' in x)
    X['Cabin'] = X.Cabin.map(lambda x : x[0] if not pd.isnull(x) else -1)
    to_dummy = ['Pclass', 'Sex', 'title', 'Embarked', 'Cabin']
    for dum in to_dummy :
        split_feature = pd.get_dummies(X[dum],prefix='split_'+dum)
        X = X.join(split_feature)
        del X[dum]
    X['Age'] = X.Age.fillna(X.Age.median())
    X['is_child'] = X.Age < 16
    to_del = ['Name', 'Ticket']
    for col in to_del : del X[col]
    return X
####
X,y=parse_model_4(train)
Xtest=parse_model_test(test)
#write.csv(Xtest,file='testrefined.csv')
#print(Xtest)
####
def compute_score(clf, X, y):
    xval = cross_val_score(clf, X, y, cv = 10)
    return np.mean(xval)


###
rf=RandomForestClassifier(n_estimators=250,max_depth=8)
rf.fit(X,y)

print("Precision de prediction en Random Forest : {:.4f} ".format(compute_score(rf, X, y)))
#Importance of variables in Random forest
#def clf_importance(X, clf):
#    import pylab as pl
#    importances = clf.feature_importances_
#    indices = np.argsort(importances)[::-1]
#    for f in range(X.shape[1]):
#        print("%d. feature : %s (%f)" % (f +1, X.columns[indices[f]],importances[indices[f]]))
        
###       
#clf_importance(X,rf)

# Predicting the Test set results
#y_pred = rf.predict(Xtest)
# Save the results
#results <- data.frame(PassengerID = titanic[892:1309,"PassengerId"], Survived = y_pred)

# Write the results to a csv file
#write.csv(results, file = 'PredictingTitanicSurvival.csv', row.names = FALSE, quote=FALSE)



Precision de prediction en Randiom Forest : 0.8406 


In [6]:
#GridSearchCV
parameters = { "n_estimators":[5,10,50,100,250],"max_depth":[2,4,8,16,32,None]}
cv = GridSearchCV(rf,parameters,cv=5)
cv.fit(X,y)
def display(results):
    print(f'Les meilleurs paramètres sont: {results.best_params_}')
    print("\n")
    mean_score = results.cv_results_['mean_test_score']
    std_score = results.cv_results_['std_test_score']
    params = results.cv_results_['params']
    for mean,std,params in zip(mean_score,std_score,params):
        print(f'{round(mean,3)} + or -{round(std,3)} for the {params}')
        
display(cv)

Les meilleurs paramètres sont: {'max_depth': 8, 'n_estimators': 250}


0.793 + or -0.012 for the {'max_depth': 2, 'n_estimators': 5}
0.777 + or -0.022 for the {'max_depth': 2, 'n_estimators': 10}
0.788 + or -0.017 for the {'max_depth': 2, 'n_estimators': 50}
0.787 + or -0.019 for the {'max_depth': 2, 'n_estimators': 100}
0.787 + or -0.019 for the {'max_depth': 2, 'n_estimators': 250}
0.815 + or -0.023 for the {'max_depth': 4, 'n_estimators': 5}
0.816 + or -0.021 for the {'max_depth': 4, 'n_estimators': 10}
0.805 + or -0.013 for the {'max_depth': 4, 'n_estimators': 50}
0.819 + or -0.015 for the {'max_depth': 4, 'n_estimators': 100}
0.819 + or -0.021 for the {'max_depth': 4, 'n_estimators': 250}
0.822 + or -0.011 for the {'max_depth': 8, 'n_estimators': 5}
0.829 + or -0.017 for the {'max_depth': 8, 'n_estimators': 10}
0.828 + or -0.019 for the {'max_depth': 8, 'n_estimators': 50}
0.832 + or -0.017 for the {'max_depth': 8, 'n_estimators': 100}
0.832 + or -0.013 for the {'max_depth': 8, 'n